# 서울시 이동인구 분석 (자치구별)

In [1]:
# # 폰트 설치
# !sudo apt-get install -y fonts-nanum
# !sudo fc-cache -fv
# !rm ~/.cache/matplotlib -rf

# # 실행 후 런타임 재시작

In [2]:
!pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 72.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 80.4 MB/s eta 0:00:00


In [3]:
!pip install folium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import os, shutil
from glob import glob
from zipfile import ZipFile

import numpy as np
import pandas as pd
import geopandas as gpd
import folium

import matplotlib.pyplot as plt
import seaborn as sns

plt.rc('font', family='NanumBarunGothic')
# sns.set(font="Malgun Gothic", #"NanumGothicCoding",
#         rc={"axes.unicode_minus":False}, # 마이너스 부호 깨짐 현상 해결
#         style='darkgrid')

import warnings
warnings.filterwarnings(action='ignore')

from google.colab.data_table import DataTable
DataTable.max_columns = 50

In [5]:
def read_file(path):
    gdf = gpd.read_file(path, encoding='cp949')
    display(df.head())
    print('shape:', df.shape)
    return gdf

def read_csv(path):
    df = pd.read_csv(path, encoding='cp949')
    display(df.head())
    print('shape:', df.shape)
    return df

def read_excel(path):
    df = pd.read_excel(path)
    display(df.head())
    print('shape:', df.shape)
    return df

# 1.Gdrive -> Colab

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# Colab 으로 데이터셋 복사
shutil.copytree('/content/drive/MyDrive/서울시내버스공모전/데이터분석/data/서울생활이동/자치구별', '/content/zips/자치구')
shutil.copytree('/content/drive/MyDrive/서울시내버스공모전/데이터분석/data/서울생활이동/master', '/content/data/master')

# 압축 해제
for file in glob('/content/zips/자치구/*'):
    ZipFile(file).extractall('/content/data/자치구')

# 2.데이터 불러오기

## 1) master

In [ ]:
sig_geo_master = gpd.read_file('data/master/move-sig-geo-master.shp')
sig_geo_master.crs = 'EPSG:5179'

In [ ]:
emd_geo_master = gpd.read_file('data/master/서울_행정동.shp')

In [11]:
sig_master = read_excel('data/master/서울생활이동데이터_자치구코드_20210907.xlsx')

,시도,시군구,name,full name
0,11000,11010,종로구,서울특별시 종로구
1,11000,11020,중구,서울특별시 중구
2,11000,11030,용산구,서울특별시 용산구
3,11000,11040,성동구,서울특별시 성동구
4,11000,11050,광진구,서울특별시 광진구


shape: (91, 4)


In [10]:
emd_master = read_excel('data/master/서울생활이동데이터_행정동코드_20210907.xlsx')

,시도,시군구,읍면동,name,full_name
0,11000,11010,1101053,사직동,서울특별시 종로구 사직동
1,11000,11010,1101054,삼청동,서울특별시 종로구 삼청동
2,11000,11010,1101055,부암동,서울특별시 종로구 부암동
3,11000,11010,1101056,평창동,서울특별시 종로구 평창동
4,11000,11010,1101057,무악동,서울특별시 종로구 무악동


shape: (1152, 5)


## 2) 자치구별 이동인구 데이터

In [26]:
paths = glob('data/자치구/*')
len(paths)

288

In [ ]:
commute_paths = [read_csv(path) for path in paths if path.split('/')[-1].split('.')[1][3:5] in ['08', '09', '18', '19']]

In [27]:
move_list = [read_csv(path) for path in paths if path.split('/')[-1].split('.')[1][3:5] in ['08', '09', '18', '19']]

'06'

In [28]:
move = read_csv(paths[0])
print(paths[0])

,대상연월,요일,도착시간,출발 시군구 코드,도착 시군구 코드,성별,나이,이동유형,평균 이동 시간(분),이동인구(합)
0,202210,일,11,11010,11010,F,0,WE,91,28.04
1,202210,일,11,11010,11010,F,0,EE,38,1762.63
2,202210,일,11,11010,11010,F,0,HE,29,362.04
3,202210,일,11,11010,11010,F,0,EH,19,28.09
4,202210,일,11,11010,11010,F,0,WH,54,27.44


shape: (1609215, 10)
data/자치구/생활이동_자치구_2022.10_11시.csv


In [14]:
# 평일만(월 ~ 금) 추출
move = move.loc[move['요일'].isin(['월', '화', '수', '목', '금'])]
move.shape

(1111899, 10)

In [16]:
# 출근시간대(07시 ~ 09시) 추출
move = move.loc[(move['도착시간'] == 8) & (move['평균 이동 시간(분)'] <= 60)]
move = move.loc[(move['도착시간'] == 9) & (move['평균 이동 시간(분)'] <= 120)]
move.shape

(0, 10)

### 1) 평일 추출

In [ ]:
move.